In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import qutip
from scipy.linalg import expm, eigh
from scipy.constants import hbar

np.set_printoptions(precision=2)

# 2 qubits with exchange interaction

Define the two-spin Pauli matrices, e.g.:

$\sigma_{1,z} = \begin{pmatrix} 1 & 0 \\ 0 & -1 \end{pmatrix} \otimes \mathcal{I}$

$\sigma_{1,x} = \begin{pmatrix} 0 & 1 \\ 1 & 0 \end{pmatrix} \otimes \mathcal{I}$

$\sigma_{1,y} = \begin{pmatrix} 0 & -i \\ i & 0 \end{pmatrix} \otimes \mathcal{I}$

In [ ]:
sigma1Z = np.kron(np.array([[1,0],[0,-1]]),np.identity(2))
sigma1X = np.kron(np.array([[0,1],[1,0]]),np.identity(2))
sigma1Y = np.kron(np.array([[0,-1j],[1j,0]]),np.identity(2))

sigma2Z = np.kron(np.identity(2),np.array([[1,0],[0,-1]]))
sigma2X = np.kron(np.identity(2),np.array([[0,1],[1,0]]))
sigma2Y = np.kron(np.identity(2),np.array([[0,-1j],[1j,0]]))

In the 1-qubit script we defined the basis of the single-spin Hamiltonian as $\{|\uparrow\rangle, |\downarrow\rangle\}$. Similarly, the basis of Hamiltonians defined using the Pauli matrices above will be $\{|\uparrow\uparrow\rangle, |\uparrow\downarrow\rangle, |\downarrow\uparrow\rangle, |\downarrow\downarrow\rangle\}$

Define the exchange Hamiltonian $\mathcal{H} = \frac{J}{2} \mathbf{S_1 S_2} = J \frac{\hbar}{4} (\sigma_{x,1}\sigma_{x,2} + \sigma_{y,1}\sigma_{y,2} + \sigma_{z,1}\sigma_{z,2})$

In [ ]:
J = 2 * np.pi

H = J * hbar/4 * (sigma1X@sigma2X + sigma1Y@sigma2Y + sigma1Z@sigma2Z)
print(H)

## Eigenstates and energies of the exchange hamiltonian

In [ ]:
energies, states = eigh(H)
print(states)
print(energies/J/hbar)

There is one lowest energy state called the singlet: 
$|S\rangle = \frac{1}{\sqrt{2}} \left( |\uparrow\downarrow\rangle - |\downarrow\uparrow\rangle \right)$

and three degenerate energy states, $J\hbar$ above, called the triplets:
$\\|T_+\rangle = |\uparrow\uparrow\rangle\\
|T_0\rangle = \frac{1}{\sqrt{2}} \left( |\uparrow\downarrow\rangle + |\downarrow\uparrow\rangle \right)\\
|T_-\rangle = |\downarrow\downarrow\rangle$

## Time evolution of the spins with exchange interaction
Initialise qubit states and form a two qubit state $| \psi \rangle = | \psi_1 \rangle \otimes | \psi_2 \rangle$

In [ ]:
psi1 = np.array([[1],[0]])
psi2 = np.array([[1],[1]])/np.sqrt(2)
psi0 = np.kron(psi1,psi2)
print(psi0)

Evolve the state in time using $| \psi_t \rangle = U(t) | \psi_0 \rangle$ with $U(t) = e^{\frac{-i H t}{\hbar}}$

In [ ]:
time = np.linspace(0,0.5,101)
psiT = []

for t in time:
    psiT.append(expm(-1j*H*t/hbar) @ psi0)

Save the expectation values of x, y and z (e.g. $\langle x_1 \rangle = \langle \psi_t | \sigma_{1,x} | \psi_t \rangle$)

In [ ]:
ev = lambda psi,oper: (psi.conj().T @ oper @ psi).real[0,0]

x1 = [ev(p, sigma1X) for p in psiT]
y1 = [ev(p, sigma1Y) for p in psiT]
z1 = [ev(p, sigma1Z) for p in psiT]

x2 = [ev(p, sigma2X) for p in psiT]
y2 = [ev(p, sigma2Y) for p in psiT]
z2 = [ev(p, sigma2Z) for p in psiT]


Plot the evolution of the states

In [ ]:
def qubitPlots(time,x,y,z):
    bsPlot = qutip.Bloch()
    for xi,yi,zi in zip(x,y,z):
        bsPlot.add_points([xi, yi, zi])
    bsPlot.render()
    bsPlot.fig.set_size_inches([25,5])
    bsPlot.show()

In [ ]:
qubitPlots(time,[x1,x2],[y1,y2],[z1,z2])

For obvious reasons, this is operation is known as SWAP

## Exchange interaction with detuned spins: controlled-phase (CZ) and controlled-rotation (CROT)

Let's now consider what happens if one of the spins is detuned in frequency by an amount $\Delta_q$. To simulate this, we need to start from the more fundamental Hubbard model, which includes the interplay between hopping $(t_0)$ and on-site $(U)$ interactions. For the case of two interacting spins, we can use the basis $\{|\uparrow\uparrow\rangle, |\uparrow\downarrow\rangle, |\downarrow\uparrow\rangle, |\downarrow\downarrow\rangle, |S_{20}\rangle\}$. The new dimension $|S_{20}\rangle$ is the state where both electrons occupy the same site. In addition, we will add terms to describe a global drive on the qubits, with amplitude $(\Omega)$ and detuning $(\Delta_{rf})$. In the above basis the Hamiltonian is:

$\mathcal{H} = \begin{bmatrix}
    \Delta_{rf} & \Omega & \Omega & 0 & 0 \\
    \Omega & \frac{\Delta_q}{2} & 0 & \Omega & t_0 \\
    \Omega & 0 & -\frac{\Delta_q}{2} & \Omega & -t_0 \\
    0 & \Omega & \Omega & \Delta_{rf} & 0 \\
    0 & t_0 & -t_0 & 0 & U 
\end{bmatrix}$

Notice that only opposing-spin states couple to the $|S_{20}\rangle$, because of the Pauli exclusion principle

In [ ]:
H11 = lambda delta_q, delta_rf, omega: delta_q/2 * (sigma1Z - sigma2Z) + delta_rf * (sigma1Z + sigma2Z) + omega * (sigma1X + sigma2X)
H20 = lambda U: U * np.identity(1)
T = lambda tunnel: np.array([[0, tunnel, -tunnel, 0]])
H = lambda delta_q, delta_rf, omega, tunnel, U: np.block([[H11(delta_q,delta_rf,omega), T(tunnel).T],
                                                          [T(tunnel), H20(U)]])

To simplify the following cells, let's add a function that simulates time evolution and outputs the expectation value for each qubit:

In [ ]:
def simulate2SpinHubbard(time, psi0, H):
    x1,y1,z1,x2,y2,z2 = ([] for _ in range(6))
    for t in time:
        psiT = expm(-1j*H*t) @ psi0
        x1.append(ev(psiT[:-1], sigma1X))
        y1.append(ev(psiT[:-1], sigma1Y))
        z1.append(ev(psiT[:-1], sigma1Z))
        x2.append(ev(psiT[:-1], sigma2X))
        y2.append(ev(psiT[:-1], sigma2Y))
        z2.append(ev(psiT[:-1], sigma2Z))
    return x1,y1,z1,x2,y2,z2

### CZ operations with pulsed exchange and no driving

In [ ]:
J = 2 * np.pi
U = 1e3
delta_rf = 0
omega = 0

### Things to try (uncomment the step that you want to test):
# 1. Check that without detuning, you get the same SWAP operation:
tunnel = np.sqrt(J*U) / 2
delta_q = 0
time = np.linspace(0,1,101)

# 2. Add detuning and check that you get the expected result (i.e., spins rotate in the xy plane at detuning frequency)
# tunnel = 0
# delta_q = 2 * np.pi
# time = np.linspace(0, 0.95, 101)

# 3. Add both, and compare what happens when control qubit (psi1) starts up vs down
# tunnel = np.sqrt(J*U) / 2
# delta_q = np.sqrt(2) * tunnel
# time = np.arange(0, np.pi, 2*np.pi/delta_q) # note that by sampling at 2*np.pi/delta_q, we are effectively moving to a rotating frame

print(H(delta_q,delta_rf,omega,tunnel,U))

In [ ]:
psi1 = np.array([[1],[0]])#/np.sqrt(2)
psi2 = np.array([[1],[1]])/np.sqrt(2)
psi0 = np.block([[np.kron(psi1,psi2)], [0]])

x1,y1,z1,x2,y2,z2 = simulate2SpinHubbard(time, psi0, H(delta_q,delta_rf,omega,tunnel,U))
qubitPlots(time,[x1,x2],[y1,y2],[z1,z2])

The controlled-phase (CP) rotations can be used to construct a CNOT gate, by applying the sequence $X_{2,\frac{\pi}{2}} - CP_\frac{\pi}{4} - Y_{2,\frac{\pi}{2}}$

### Driven CROT operations with fixed exchange 

#### Control of detuned qubits

Let's now introduce the driving terms to realise CROT operations. Let's first turn off the exchange and confirm that we can drive rotations on the spins:

In [ ]:
delta_rf = 0
omega = np.pi
delta_q = 0
tunnel = 0

time = np.linspace(0,0.95,101)
psi1 = np.array([[1],[0]])
psi2 = np.array([[1],[0]])
psi0 = np.block([[np.kron(psi1,psi2)], [0]])
x1,y1,z1,x2,y2,z2 = simulate2SpinHubbard(time, psi0, H(delta_q,delta_rf,omega,tunnel,U))
qubitPlots(time,[x1,x2],[y1,y2],[z1,z2])

Both spins are degenerate and are being driven by a global resonant field, so they rotate together. 

Now try setting a qubit detuning (e.g. `delta_q = 100`) and see what happens. What do you need to do now to control your spins?

#### What happens when we add exchange coupling?

To understand this, we will apply a drive for a fixed period of time and measure each qubit state at the end of the drive. We can then make plot of this measurement as a function of drive frequency and exchange coupling.

Play around with different combinations of initial qubit states, to try to understand what happens when the exchange is turned on.

In [ ]:
U = 1e4
delta_rf = np.linspace(-100,100,101)
omega = np.pi
delta_q = 100
tunnel = np.linspace(0,500,101)
time = 10.5
psi2 = np.array([[1],[0]])
psi1 = np.array([[1],[0]])
# time = 10.25
# psi1 = np.array([[1],[1j]])/np.sqrt(2)
# psi2 = np.array([[1],[1j]])/np.sqrt(2)
psi0 = np.block([[np.kron(psi1,psi2)], [0]])

z1,z2 = (np.zeros([len(delta_rf),len(tunnel)]) for _ in range(2))
for id,d in enumerate(delta_rf):
    for it,t in enumerate(tunnel):
        psiT = expm(-1j*H(delta_q,d,omega,t,U)*time) @ psi0
        z1[id,it] = ev(psiT[:-1], sigma1Z)
        z2[id,it] = ev(psiT[:-1], sigma2Z)
        
fig, axs = plt.subplots(1,2)
axs[0].pcolormesh(tunnel, delta_rf, z1, shading="nearest")
im = axs[1].pcolormesh(tunnel, delta_rf, z2, shading="nearest")
fig.colorbar(im, ax=axs[1])
axs[0].set_title("Qubit 1 measurement")
axs[1].set_title("Qubit 2 measurement")
axs[0].set_xlabel("$t_0$")
axs[1].set_xlabel("$t_0$")
axs[0].set_ylabel("Detuning")

When we turn on the coupling, we get separate qubit control frequencies, that depend on the state of the other qubit. How do we use this to make a CNOT gate?